In [21]:
from peewee import *
from playhouse.postgres_ext import *
import datetime

db = PostgresqlDatabase('test', user='postgres', password='password', host='localhost', port=5432)


In [38]:
import logging
logger = logging.getLogger('peewee')
logger.addHandler(logging.StreamHandler())
logger.setLevel(logging.DEBUG)

In [22]:
class BaseModel(Model):
    class Meta:
        database = db

class User(BaseModel):
    username = TextField()

class Tweet(BaseModel):
    content = TextField()
    code = ArrayField(CharField, null=True)
    cat = CharField(null=True)
    timestamp = DateTimeField(default=datetime.datetime.now)
    user = ForeignKeyField(User, backref='tweets')

class Favorite(BaseModel):
    user = ForeignKeyField(User, backref='favorites')
    tweet = ForeignKeyField(Tweet, backref='favorites')

In [23]:
db.drop_tables([User, Tweet, Favorite])
db.execute_sql('drop AGGREGATE if EXISTS array_concat_agg(anycompatiblearray);')


<cursor object at 0x000001F35950DB60; closed: 0>

In [24]:
def populate_test_data():
    db.create_tables([User, Tweet, Favorite])
    db.execute_sql('CREATE or replace AGGREGATE array_concat_agg(anycompatiblearray) (   SFUNC = array_cat,   STYPE = anycompatiblearray );')
    data = (
        ('huey', ('meow', 'hiss', 'purr'), ('1001','1002'), 'cat1'),
        ('mickey', ('woof', 'whine'), ('1003'), 'cat2'),
        ('zaizee', ('hello', 'greet'), ('1005','1006', '1007'), 'cat1')
    )
    for username, tweets, code, cat in data:
        user = User.create(username=username)
        for tweet in tweets:
            print(f"Tweet = {tweet}, code = {code}")
            Tweet.create(user=user, content=tweet, code=code, cat=cat)

    # Populate a few favorites for our users, such that:
    favorite_data = (
        ('huey', ['whine']),
        ('mickey', ['purr']),
        ('zaizee', ['meow', 'purr']))
    for username, favorites in favorite_data:
        user = User.get(User.username == username)
        for content in favorites:
            tweet = Tweet.get(Tweet.content == content)
            Favorite.create(user=user, tweet=tweet)

In [25]:
populate_test_data()

Tweet = meow, code = ('1001', '1002')
Tweet = hiss, code = ('1001', '1002')
Tweet = purr, code = ('1001', '1002')
Tweet = woof, code = 1003
Tweet = whine, code = 1003
Tweet = hello, code = ('1005', '1006', '1007')
Tweet = greet, code = ('1005', '1006', '1007')


In [26]:
[t for t in Tweet.select().dicts()]

[{'id': 1,
  'content': 'meow',
  'code': ['1001', '1002'],
  'cat': 'cat1',
  'timestamp': datetime.datetime(2024, 7, 6, 9, 12, 0, 839737),
  'user': 1},
 {'id': 2,
  'content': 'hiss',
  'code': ['1001', '1002'],
  'cat': 'cat1',
  'timestamp': datetime.datetime(2024, 7, 6, 9, 12, 1, 38856),
  'user': 1},
 {'id': 3,
  'content': 'purr',
  'code': ['1001', '1002'],
  'cat': 'cat1',
  'timestamp': datetime.datetime(2024, 7, 6, 9, 12, 1, 237703),
  'user': 1},
 {'id': 4,
  'content': 'woof',
  'code': ['1', '0', '0', '3'],
  'cat': 'cat2',
  'timestamp': datetime.datetime(2024, 7, 6, 9, 12, 1, 655465),
  'user': 2},
 {'id': 5,
  'content': 'whine',
  'code': ['1', '0', '0', '3'],
  'cat': 'cat2',
  'timestamp': datetime.datetime(2024, 7, 6, 9, 12, 1, 857772),
  'user': 2},
 {'id': 6,
  'content': 'hello',
  'code': ['1005', '1006', '1007'],
  'cat': 'cat1',
  'timestamp': datetime.datetime(2024, 7, 6, 9, 12, 2, 251601),
  'user': 3},
 {'id': 7,
  'content': 'greet',
  'code': ['1005', '

In [27]:
[t for t in Tweet.select(SQL('array_concat_agg(distinct code) as code')).group_by(Tweet.cat).dicts()]

[{'code': ['1001', '1002', '1005', '1006', '1007']},
 {'code': ['1', '0', '0', '3']}]

In [11]:
[t for t in Tweet.select(SQL('array_agg(distinct code) as code')).group_by(Tweet.cat).dicts()]

DataError: cannot accumulate arrays of different dimensionality
